In [1]:
import numpy as np
import pandas as pd
import sklearn
# import matplotlib.pyplot as plt
# import category_encoders as ce
import sklearn.metrics
from sklearn.linear_model import LogisticRegression
import random
from sklearn import svm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

train_data = train_data.drop(columns=['id',
                                      'addr_state',
                                      'application_type',
                                      'earliest_cr_line',
                                      'emp_title',
                                      'home_ownership',
                                      'initial_list_status',
                                      'issue_d',
                                      'purpose',
                                      'sub_grade',
                                      'title',
                                      'verification_status',
                                      'zip_code',
                                      'mort_acc',
                                      'emp_length',
                                      'revol_util'])


In [3]:
grade_2_index = {"A": 0,
                 "B": 1,
                 "C": 2,
                 "D": 3,
                 "E": 4, 
                 "F": 5,
                 "G": 6}

# Get features: income, dti
X = []
Y = []
for index, row in train_data.iterrows():
    idx = grade_2_index[row['grade']]
    feature = [0 for _ in range(7)]
    feature[idx] = 1
    X.append(feature)

    label = row['loan_status']
    if label == 'Fully Paid':
        Y.append(0)
    else:
        Y.append(1)

NEURAL NETWORKS

In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.X = features
        self.Y = labels

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.Y)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        # print(f"SHAPE {self.X.shape}, {self.Y.shape}")
        # print(f"({self.X[index]}, {self.Y[index]})")
        y = torch.reshape(self.Y[index], (1,))
        return self.X[index], y

In [5]:
# fix class imbalance
X_balanced = []
Y_balanced = []
for k in range(len(X)):
    if Y[k] == 1:
        for _ in range(3):
            X_balanced.append(X[k])
            Y_balanced.append(Y[k])
    X_balanced.append(X[k])
    Y_balanced.append(Y[k])

print(len(X_balanced))

287922


In [6]:
DLmodel = nn.Sequential(
    nn.Linear(7, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)
optimizer = torch.optim.Adam(DLmodel.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

X_tensor = torch.FloatTensor(X_balanced)
Y_tensor = torch.FloatTensor(Y_balanced)
dataset = CustomDataset(X_tensor, Y_tensor)
train_loader = torch.utils.data.DataLoader(dataset, shuffle=True)


In [7]:
DLmodel.train()

for epoch in range(5):
    correct = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()
        # Forward pass
        output = DLmodel(data)
        # output = torch.reshape(output, (1,))
        # print('FUCS')
        # print(output)
        # print(type(output))
        # print(type(label))
        # print(label)
        # Calculate loss
        # output_tensor = torch.FloatTensor(output)
        # label_tensor = torch.reshape(output, (1, 1))
        # print(f'SHAPES {output.shape}, {label_tensor.shape}')
        # print(f'tensors {output}, {label_tensor}')
        loss = nn.functional.binary_cross_entropy_with_logits(output, label)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

        # to calculate accuracy
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(label.view_as(pred)).sum().item()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f. Training Accuracy: (%.4f)' % (epoch + 1,  loss.item(), 100*correct/len(train_loader.dataset)))

In [ ]:
pred = DLmodel(X_tensor)
# print(pred)
# prob_preds = []
# for j in range(len(pred)):
#     prob_preds.append(pred[j][1])
print("SCORE")
print(DLmodel(torch.FloatTensor([1,0,0,0,0,0,0])))
sklearn.metrics.roc_auc_score(Y_balanced, pred.detach().numpy())

SCORE
tensor([2.1910e-13], grad_fn=<SigmoidBackward>)


0.6343040086159206

In [ ]:
predictions = {"id": [], "loan_status": []}
for index, row in test_data.iterrows():
    credit_score = row["grade"]
    idx = grade_2_index[credit_score]
    feature = [0 for _ in range(7)]
    feature[idx] = 1
    feature = torch.FloatTensor(feature)
    # print(feature)
    # print(type(feature))
    prediction = DLmodel(feature).item()
    # print(prediction)
    predictions["id"].append(row['id'])
    predictions["loan_status"].append(prediction)

In [ ]:
predictions_df = pd.DataFrame(data=predictions)
predictions_df.to_csv("neural_net_balanced_predictions", index=False)
assert False

AssertionError: 

LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression(C=0.001)
model.fit(X, Y)
pred = model.predict_proba(X)
prob_preds = []
for j in range(len(pred)):
    prob_preds.append(pred[j][1])
print("SCORE")
print(model.predict_proba([[1,0,0,0,0,0,0]])[0][1])
sklearn.metrics.roc_auc_score(Y, prob_preds)

SCORE
0.08122640241594924


0.6545479517428292

In [ ]:
predictions = {"id": [], "loan_status": []}
for index, row in test_data.iterrows():
    credit_score = row["grade"]
    idx = grade_2_index[credit_score]
    feature = [0 for _ in range(7)]
    feature[idx] = 1
    prediction = model.predict_proba([feature])[0][1]
    predictions["id"].append(row['id'])
    predictions["loan_status"].append(prediction)

In [ ]:
predictions_df = pd.DataFrame(data=predictions)
predictions_df.to_csv("LLLLL_predictions", index=False)

In [ ]:
# # Get features: income, dti
# predictions = []
# ground_truths = []
# indicator = True
# X = []
# NAN_count = 0
# for index, row in train_X.iterrows():
#     nan_indices = pd.Series.tolist(row.isnull())

#     features =pd.Series.to_list(row)
#     for i in range(len(nan_indices)):
#         if nan_indices[i] == True:
#             features[i] = 0
#             NAN_count += 1
    
#     # convert percent to decimal
#     features[2] = float(features[2].strip('%'))/100

#     X.append(features)
# print(f'FOUND NANCOUTN {NAN_count}')
# print(f"entries {train_X.shape}")
# Y = []
# for label in train_y.values:
#     if label == 'Fully Paid':
#         Y.append(0)
#     else:
#         Y.append(1)

In [ ]:
# Get features: income, dti
# X = []
# for index, row in train_X.iterrows():
#     features = [row['annual_inc'], row['dti']]
#     X.append(features)

# Y = []
# for label in train_y.values:
#     if label == 'Fully Paid':
#         Y.append(0)
#     else:
#         Y.append(1)
# print(np.unique(Y))

In [ ]:
# model = svm.SVC(kernel='rbf') # consider parameter probability=True

# count = 0
# for j in range(len(Y)):
#     if Y[j] == 0:
#         count += 1
# print(f'CONT {count}')
# X_sample = []
# Y_sample = []
# budget0 = 17000
# budget1 = 15000
# for i in range(len(Y), -1, -1):
#     if Y[j] == 0:
#         budget0 -= 1
#     else:
#         budget1 -= 1
#     X_sample.append(X[j])
#     Y_sample.append(Y[j])
#     if budget0 == 0 or budget1 == 0:
#         break

# print(f'YSAMPLE UNIQUE {np.unique(Y_sample)}')
# print(len(X_sample))
# model.fit(X_sample, Y_sample)
# in_sample_preds = model.predict(X_sample)
# print(np.unique(in_sample_preds))
# # print(in_sample_preds)

# print("SCORE")
# sklearn.metrics.roc_auc_score(Y_sample, in_sample_preds)

In [ ]:
# model = svm.SVC(kernel='linear') # consider parameter probability=True
# Y = train_y.values.tolist()
# model.fit(X, Y)
# in_sample_preds = model.predict(X)

# sklearn.metrics.roc_auc_score(Y, in_sample_preds)

In [ ]:
# model = svm.SVC(kernel='rbf') # consider parameter probability=True

# X_sample = []
# Y_sample = []
# for j in range(len(X)):
#     p = random.random()
#     if p <= 0.4:
#         X_sample.append(X[j])
#         Y_sample.append(Y[j])
# print(f'YSAMPLE UNIQUE {np.unique(Y_sample)}')
# print(len(X_sample))
# model.fit(X_sample, Y_sample)
# in_sample_preds = model.predict(X_sample)
# print(np.unique(in_sample_preds))
# # print(in_sample_preds)

# print("SCORE")
# sklearn.metrics.roc_auc_score(Y_sample, in_sample_preds)